# Quantenkryptographie und -teleportation
In diesem Notebook sollen einige Aspekte zum Thema Informationsübertragung mit Quantencomputern untersucht werden.

## Vorbereitungen
Wie in jedem Jupyter Notebook werden zunächst die benötigten Pakete geladen und einige Voreinstellungen getroffe. (Siehe hierzu auch ["Einstieg in Qiskikt"](https://github.com/MarkEich96/QuantencomputingWiSe23-24/blob/main/Einstieg%20in%20Qiskit.ipynb))

__Pakete__

In [ ]:
import numpy as np
import qiskit
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister)
from qiskit import(execute, Aer, BasicAer)
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram

__Anzeigen von Grafiken__

In [ ]:
%matplotlib inline

__Einstellen der Simulationssoftware__

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

## Dichte Kodierung

Bei der Behandlung des ["Problems von Deutsch"](https://github.com/MarkEich96/QuantencomputingWiSe23-24/blob/main/Das%20Problem%20von%20Deutsch.ipynb) und verwandten Algorithmen, haben wir bereits gesehen, dass Quantencomputer aufgrund der Superposition der Zustände in der Lage sind, Aufgaben mit weniger Aufrufen zu lösen als ein klassischer Computer. Es scheint also, als könnte ein Quantencomputer mehr Informationen auf einmal verarbeiten. Damit stellt sich die Frage, ob es auch möglich ist mehr Informationen als mit einem klassischen Computer zu übertragen. 

### Verschränkte Qubits
Alice und Bob wollen mit Hilfe von Qubits eine von vier Optionen übertragen und dabei nur ein einziges Qubit versenden. Sie treffen sich und präperieren Ihre beiden Qubits.

__Aufgabe__: Betrachte den Quantenschaltkreis, den der unten stehende Code erzeugt. In welchem Zustand befinden sich die Qubits nach Ausführen des Schaltkreises? Warum müssen sich Alice und Bob treffen, um diesen Schaltkreis auszuführen?

In [ ]:
# Erstelle ein Quantenregister mit Alices Bit
alice = QuantumRegister(1, name = 'a')
# Erstelle ein Quantenregister mit Bobs Bit
bob = QuantumRegister(1, name = 'b')

# Füge die beiden Bits zu einem Schaltkreis zussammen
circuit = QuantumCircuit(bob, alice)

# Initialisiere die beiden Bits in den Zuständen |0>
circuit.reset(range(2))

# Wende die Hadamard-Transformation auf Alices Bit an
circuit.h(1)

# Wende das CNOT-Gatter mit ALices Bit als Kontrollbit an
circuit.cx(1, 0)

# Zeichne den Schaltkreis
circuit.draw(output = 'mpl')

### Operation auf einem Qubit

Nun begibt sich Alice mit ihrem Qubit an einen von Bob weit entfernten Ort. Alice entscheidet sich eine von vier Operationen auf ihr Qubit auszuführen. Die vier Operationen, die ihr zur Verfügung stehen sind $I$, $X$, $Y$ und $Z$. (Siehe auch [Wikiversity-Artikel Qubits](https://de.wikiversity.org/wiki/Kurs:Quantencomputing/Qubits))

In [ ]:
def alice_change_bit(Circuit, operation = "I"):
    if operation == "I":
        Circuit.id(1)
    elif operation == "X":
        Circuit.x(1)
    elif operation == "Y":
        Circuit.y(1)
    elif operation == "Z":
        Circuit.z(1)
    else:
        print("Da ist etwas schief gelaufen in alice_change_bit")

__Aufgabe__: In welchen Zuständen befindet sich das Register nach dem Ausführen der einzelnen Operationen.

### Extrahieren der Informationen

Alice schickt ihr Qubit an Bob. Dieser möchte aus Alices und seinem Qubit nun herausfinden, welche Operation Alice angewandt hat.

__Aufgabe__: Wie lässt sich aus dem Zustand $|\Phi^+\rangle$ wieder der Zustand $|00\rangle$ gewinnen? Was passiert, wenn die gleichen Operationen auf die anderen drei Zustände, die sich nach dem Ausführen einer Operation auf Alice' Bit ergeben, angewandt werden?

### Implementieren des Vorgehens

__Aufgabe__: Vervollständige das unten stehende Code-Gerüst, um den vollständigen Schaltkreis für das hier diskutierte Kodierungsverfahren von Informationen zu skizzieren. 

In [ ]:
# Erstelle einen Quantenschaltkreis mit Qubits für Alice und Bob
# und 2 klassischen Bits
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')
classical = ClassicalRegister(2, name ='c')

circuit = QuantumCircuit(bob, alice, classical)

# Generiere den Anfangszustan |Phi+>
circuit.reset(range(2))
circuit.h(1)
circuit.cx(1, 0)

# Ergänze den Rest des Codes mit den oben besprochenen Operationen



# Zeichne den Schaltkreis
circuit.draw(output = 'mpl')

### Interpretieren des Informationsaustausch

Das hier vorgestellet Verfahren wird als _dichte Codierung_ bezeichnet.

__Aufgabe__: Wie viele klassische Bits müsste Alice an Bob senden, um eine von vier Optionen zu übermitteln? Stelle Dir vor, Alice und Bob befinden sich niemals am selben Ort? Wie viele Qubits müssen sie dann übertragen, um diesen Algorithmus auszuführen?

### Ausprobieren des Verfahrens

Mit dem folgenden Code, kannst Du Deinen Schaltkreis ausprobieren.

In [ ]:
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

## Das No-Cloning-Theorem
Eine weitere Operation, die ein klassischer Computer mit Informationen vornehemen kann, ist es diese zu kopieren. Damitk kann die Frage aufkommen, ob es möglich ist Qubits zu kopieren. 

Dazu können zwei Qubits in den Zuständen $|\psi\rangle|\phi\rangle$ betrachtet werden. Wenn ein Rechner in der Lage sein soll, den Quantenzustand $|\psi\rangle$ zu kopieren, dann müsste es einen unitären Operator $U_C$ mit der Eigenschaft
$$U_C|\psi\rangle|\phi\rangle=|\psi\rangle|\psi\rangle$$
geben. 

__Aufgabe__: Betrachte zwei Anfangszustände $|\psi_{1, 2}\rangle|\phi\rangle$ und bestimme deren Skalarprodukt nachdem auf beide $U_C$ angewandt wurde. Welche beiden Ausdrücke lassen sich finden? Warum ist es damit unmöglich beliebige Quantenzustände zu kopieren?

## Quantenteleportation
Wenn nun nach dem No-Cloning-Theorem ein Copy und Paste unmöglich ist, könnte es wenigstens die Hoffnung geben, dass ein Cut and Paste durchgeführt werden kann. Das ein Qubit also an einer Stelle dem Schaltkreis entnommen wird und an anderer Stelle wieder eingesetzt wird.

### Vorbereitungen

Bob möchte Alice ein Qubit schicken. Durch ein vorheriges Treffen konnten sie mit dem untenstehenden Schaltkreis ein verschränktes Paar an Qubits erzeugen, die im Zustand $|a\rangle|b\rangle=|\Phi^+\rangle$ sind.

In [ ]:
# Erstelle Quantenregister für Alice und Bob
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')

# Erstelle einen Quantenschaltkreis
circuit = QuantumCircuit(bob, alice)

# Verschränke die Qubits in den Zustand |Phi+>
circuit.reset(range(2))
circuit.h(1)
circuit.cx(1, 0)

# Skizziere den Schaltkreis
circuit.draw(output = 'mpl')

### Operationen am zu übermittelnden Bit

Nun präperiert Bob ein Qubit $|\psi\rangle$ in einen beliebigen Zustand 
$$|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$$
Er verschränkt dieses anschließend mit seinem Qubit $|b\rangle$, das mit Alices Qubit $|a\rangle$ verschränkt ist. Dazu benutzt er den Schaltkreis, den der folgende Code erzeugt.

In [ ]:
# Erstelle Quantenregister für Alice und Bob
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')
psi = QuantumRegister(1, name = '\psi')

# Erstelle einen Quantenschaltkreis
circuit = QuantumCircuit(psi, bob, alice)

# Verschränke die Qubits in den Zustand |Phi+>
circuit.reset(range(1, 3))
circuit.h(2)
circuit.cx(2, 1)

# Räumliche Trennung von Alice und Bob
circuit.barrier()

# Initialiseren von |Psi> (al, bet und init erzeigen das schön aussehende lila Gatter)
circuit.reset(0)
al = qiskit.circuit.Parameter(r'$\alpha$')
bet = qiskit.circuit.Parameter(r'$\beta$')
init = qiskit.circuit.Gate(name = r'$|\psi\rangle$', num_qubits = 1, params = [al, bet])
circuit.append(init, [0])

# Verschränke |Psi> mit |b>
circuit.cx(0, 1)
circuit.h(0)


# Skizziere den Schaltkreis
circuit.draw(output = 'mpl')

__Aufgabe__: In welchem Zustand befindet sich das Register $|R\rangle=|a\rangle|b\rangle|\psi\rangle$ nach dem Ausführen des Schaltkreises?

### Übertragen der Informationen
Nun misst Bob seine beiden Bits und schreibt sich die Ergebnisse auf. 

__Aufgabe__: Erstelle eine Tabelle für die möglichen Messergebnisse von $b$ und $\psi$. In welchem Zustand befindet sich dann das Bit von Alice?

Bob schickt seine Messunegn über einen klassischen Weg an Alice. 

__Aufgabe__: Welche Operationen muss sie auf ihr Bit anwenden, um es in den ursrpünglichen Zustand $|\psi\rangle=\alpha|0\rangle+\beta |1\rangle$ umzuwandeln?

### Implementieren des Algorithmus

__Aufgabe__: Verwende den bedingten Bitflip $X$ ``cx`` und das bedingte $Z$ ``cz``, um das Verfahren im unten stehenden Code-Gerüst zu implementieren.

In [ ]:
# Erstelle Quantenregister für Alice und Bob
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')
psi = QuantumRegister(1, name = '\psi')

# Ein klassisches Register für die Messung
classical = ClassicalRegister(1, name = 1)

# Erstelle einen Quantenschaltkreis
circuit = QuantumCircuit(psi, bob, alice, classical)

# Verschränke die Qubits in den Zustand |Phi+>
circuit.reset(range(1, 3))
circuit.h(2)
circuit.cx(2, 1)

# Räumliche Trennung von Alice und Bob
circuit.barrier()

# Initialiseren von |Psi> 
# Hier werden zufällige Parameter für Psi gewählt
circuit.reset(0)
th, phi = np.pi*np.random.rand(), 2*np.pi*np.random.rand()
circuit.u(th, phi, 0, 0)

# Verschränke |Psi> mit |b>
circuit.cx(0, 1)
circuit.h(0)

circuit.barrier()

# Ergänze hier die fehlenden Operationen, um |psi> in das Bit |a> zu übertragen




# Um zu prüfen ob der Algoritmus funktioniert hat, soll |a> auf |0> gedreht werden
# Mit welchen Parametern wird diese Drehung durchgeführt?
circuit.u(th, np.pi, np.pi-phi, 2)

# Messe Bit |a> um zu prüfen, ob sich stets 0 ergibt
circuit.measure(2, 0)

# Skizziere den Schaltkreis
circuit.draw(output = 'mpl')

### Ausführen des Schaltkreieses
Mit dem untenstehenden Code, kannst Du den Schaltkreis ausprobieren.

In [ ]:
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

## Quantenrepeater

Sowohl bei der Dichten Kodierung als auch bei der Quantenteleportation mussten Alice und Bob über ein Paar verschränkter Qubits verfügen. Was kann aber getan werden, wenn Alice und Bob sich nie getroffen haben und auch sonst in keinem Kontakt zueinander standen? Ist es möglich durch eine Person zwischen den beiden einen _Quantenkanal_ aufzubauen? Der Hintergrund ist noch ein anderer: Je weiter ein verschränktes Qubit übertragen wird, desto anfälliger wird es dafür, dass durch Störeinflüsse die Verschränkung endet. (Dieser Effekt wächst exponentiell mit dem Abstand) Ist es möglich mit geringerem Ressourcenaufwand Quantenkanäle aufzubauen?

### Vorbereitung

Alice und Bob stehen in keinem direkten Kontakt. Zwischen Ihnen befindet sich Charlie und ist in der Lage, beiden ein Qubit zukommen zu lassen. Charlie bereitet die beiden Qubits $a$ und $b$ mit dem unten stehenden Quantenschaltkreis vor.

In [ ]:
# Erzeuge die Qubits a, b und 2 Qubits für Charlie
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')
charlie = QuantumRegister(2, name = 'c')

# Fasse die Qubits zu einem Schalkreis zusammen
circuit = QuantumCircuit(bob, charlie, alice)

# Initialisiere die Qubits
circuit.reset(range(4))

# Verschränke die Qubits von Charlie mit den Qubits für Alice und Bob
circuit.h(range(1, 3))
circuit.cx(1, 0)
circuit.cx(2, 3)

# Zeichne den Schaltkreis
circuit.draw(output = 'mpl')

__Aufgabe__: In welchem Zustand befindet sich das Register $|R\rangle=|a\rangle|c_1\rangle|c_0\rangle|b\rangle=|ac_1c_0b\rangle$ nach dem Ausführen dieses Schaltkreises?

### Übertragen der Qubits

Nun verschickt Charlie die beiden Qubits an Alice und Bob. Nach dem diese die Qubits erhalten haben führt Charlie auf die Qubits, die sie behalten hat den untenstehenden Schaltkreis aus.

In [ ]:
# Schaltkreis mit Charlies Qubits
charlie = QuantumRegister(2, name = 'c')
circuit = QuantumCircuit(charlie)

circuit.cx(1, 0)
circuit.h(1)

circuit.draw(output = 'mpl')

__Aufgabe__: In welchem Zustand befindet sich das Register nach dem Charlie diesen Schaltkreis angewendet hat?

### Verschränkung erzeugen

Nun misst Charlie die beiden Qubits $c_0$ und $c_1$. 

__Aufgabe__: Erstelle eine Tabelle mit den Zuständen $|\psi\rangle=|a\rangle|b\rangle=|ab\rangle$ für die möglichen Messergebnisse von Charlie.

### Anpassen der Zustände
In den beiden oben vorgestellten Methoden liegen die verschränkten Qubits immer im Zustand $|\Phi^+\rangle=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$ vor. Charlie teilt Bob die Messergebnisse mit.

__Aufgabe__: Welche Opertationen muss Bob anwenden, damit die Qubits $|a\rangle|b\rangle$ am Ende im Zustand $|\Phi^+\rangle$ sind?

### Implementierung des Algorithmus
Ein solches Herstellen von verschränkten Quantenpaaren (Quantenkanälen) über eine Mittelsperson wird als [_Quantenrepeater_](https://de.wikipedia.org/wiki/Quantenrepeater) bezeichnet. Weil zwei zuvor miteinander unverschränkte Qubits durch das Verschränken zweier anderer Qubits und das anschließende Messen, miteinander verschränkt werden, wird diese Methode auch als _Entangelment swapping_ bezeichnet.

__Aufgabe__: Implementiere das Verfahren in das untenstehende Code-Gerüst.

In [ ]:
# Erzeuge die Qubits a, b und 2 Qubits für Charlie
alice = QuantumRegister(1, name = 'a')
bob = QuantumRegister(1, name = 'b')
charlie = QuantumRegister(2, name = 'c')

classical = ClassicalRegister(2, name ='m')

# Fasse die Qubits zu einem Schalkreis zusammen
circuit = QuantumCircuit(bob, charlie, alice, classical)

# Implementiere den Quantenrepeater


# Zeichne den Schaltkreis
circuit.draw(output = 'mpl')

### Schaltkreis des Schaltkreises
Mit dem unten stehenden Code kannst Du prüfen, ob der Schaltkreis funktioniert.

In [ ]:
# Diese Codezeilen erweitern den Schaltkreis,
# so dass der Zustand |phi+> auf |00> gedreht wird
circuit.barrier()

circuit.cx(0, 3)
circuit.h(0)

circuit.barrier()

circuit.measure(0, 0)
circuit.measure(3, 1)

In [ ]:
# Ausführen des Schaltkreises
job = execute(circuit, simulator, shots = 1000)
results = job.result()
plot_histogram(results.get_counts())

## Schlüsselerzeugung mit Qubits

Alice und Bob wollen Nachrichten austauschen, die nur sie lesen können. Dazu möchten sie die Nachrichten verschlüsseln. Es gibt unzählige Verfahren Nachrichten zu ver- und entschlüsseln und dieses Feld der Mathematik heißt [_Kryptographie_](https://de.wikipedia.org/wiki/Kryptographie). Alice und Bob würden ihre Nachrichten nicht verschlüsseln, wenn sie nicht fürchten müssten, dass eine weitere Person Eve versuchen würde die Nachricht abzufangen. Daher kommt die Frage auf, ob Alice und Bob Quantencomputer benutzen könnten, um einen Geheimschlüssel zu erzeugen, den Eve nicht in die Hände bekommen kann.

### Alice vorgehen
Alice beginnt damit zwei Qubits $|a_1\rangle$ und $|a_0\rangle$ im Zustand $|a_1a_0\rangle=|00\rangle$ zu erzeugen. Sie erzeugt damit zunächst zwei Zufallsbits $a_1', a_0'\in\{0, 1\}$ mit je $50\%$-iger Wahrscheinlichkeit für beide Ausgänge. Ist $a_1'=1$ so wendet sie auf $|a_0\rangle$ die Hadamard-Transformation an. 

__Aufgabe__: Implementiere Alices vorgehen im untenstehenden Code-Gerüts.

In [ ]:
# Erstelle ein Register mit Alice Qubits
alice = QuantumRegister(2, name = 'a')

# Erstelle einen Quantenschaltkreis mit zwei klassischen Bits zum Messen
classical = ClassicalRegister(2, name = '$a^{\prime}$')
circuit = QuantumCircuit(alice, classical)

# Implementiere das oben beschriebene Verfahrens


# Zeichne den Schaltkreis
circuit.draw(output='mpl')

__Aufgabe__: Fertige eine Tabelle mit den möglichen Zuständen von $|a_0\rangle$ nach dem Ausführen des Algorithmus für die möglichen Kombinationen von $a_0'$ und $a_1'$ aus.

### Bobs vorgehen
Alice schickt nun das Bit $|a_0\rangle$ an Bob, hält das Bit $a_1'$ aber bei sich und geheim. Bob ist ebenfalls im Besitzt eines Qubits $|b_1\rangle$ und erzeugt damit ein Zufallsbit $b_1'$. Ist $b_1'=1$, so wendet er die Hadamard-Transformation auf $|a_0\rangle$ an. 

__Aufgabe__: Implementiere Bobs vorgehen im untenstehenden Code-Gerüst.

In [ ]:
# Erstelle Bobs Qubits. Qubit 0 erhält Bob von Alice
bob = QuantumRegister(2, name = 'b')

# Fasse alles zu einem Schaltkreis mit zwei klassischen Bits zum Messen zusammen
classical = ClassicalRegister(2, name = '$b^{\prime}$')
circuit = QuantumCircuit(bob, classical)

# Erstellen eines Extragatters, um das Empfangen von |a0> zu symbolisieren
recive = qiskit.circuit.Gate(name = r'$|a_0\rangle$', num_qubits = 1, params = [])
circuit.append(recive, [0])
circuit.barrier()

# Implementiere Bobs vorgehen


# Zeichne den Schaltkreis
circuit.draw(output = 'mpl', plot_barriers = False)

__Aufgabe__: Erstelle eine Tabelle mit den möglichen Werten für $a_1'$, $a_0'$, $b_1'$ und $b_0'$. Ist kein eindeutiger Wert zuzuordnen, markiere dies mit $X$. Was folgt daraus für das Erstellen von Schlüsseln?

### Eve's Strategie
Eve möchte gerne den Schlüssel, den Alice und Bob erzeugen abhören und so auch in Besitz des Schlüssels kommen. Zur Vereinfachung werden wir uns hier nur einer Strategie widmen, es gibt aber noch viele andere, deren Ausgang ähnlich ist. 

Am besten wäre es, wenn Eve das Qubit $|a_0\rangle$ abfangen könnte, dieses Kopieren und an Bob weiterschicken. 

__Frage__: Warum geht das nicht?

Stattdessen muss die das Qubits $|a_0\rangle$ messen. Sie weiß genau so wenig wie Bob, ob sie vorher die Hadamard-Matrix anwenden muss, oder nicht. Also geht sie genau wie Bob vor und erstellt ein Zufallsbit $e_1'$ und wendet entsprechend vor der Messung die Hadamard-Matrix an. Nach dem Messen schickt sie $|a_0\rangle$ an Bob weiter. Ist $e_1'=1$, so muss sie nach dem Messen und vor dem Verschicken nochmal die Hadmard-Matrix anwenden.

__Aufgabe__: fertige eine Tabelle für die möglichen Kombinationen von $a_1'$, $a_0'$, $e_1'$, $e_0'$, $b_1'$ und $b_0'$ an, bei denen Alice und Bob ihre Bits für den Schlüssel nicht verwerfen würden an. Was fällt Dir auf?

__Aufgabe__: Bob schickt nu nicht nur das Bit $b_1'$ sondern auch $b_0'$ an Alice. Wie können sie so herausfinden, ob sie belauscht wurden? Wie wahrscheinlich ist es, dass Eve auffliegt?

__Anmerkung__: Es gibt noch viele andere Strategien, die verfolgt werden können doch bei allen zeigt sich grob folgender Zusammenhang: Je mehr Informationen Eve aus dem Bit entnimmt, desto höher die Chance, dass sie auffliegen kann, da sie das Qubit zu sehr verändert. 

### Das BB84-Protokoll
1984 entwickelten Charles Benneet und Gilles Brassard das folgenden [Protokoll](https://de.wikipedia.org/wiki/Quantenschl%C3%BCsselaustausch):
  1. Alice erzeugt $2m$ Zufallsbits $a_{01}',\dots, a_{0m}'$ und $a_{11}', \dots a_{1m}'$
  1. Ist das Bit $a_{1k}'=1$ so wendet sie $H$ auf $|a_{0k}\rangle$ an.
  1. Sie hält die Bits $a_{1k}'$ geheim und schickt die Bits $a_{0k}'$ an Bob
  1. Bob erzeugt $m$ Zufallsbits $b_{10}',\dots, b_{1m}'$. Ist $b_{1k}'=1$ so wendet er $H$ auf $|a_{0k}\rangle$ an. Anschließend misst er die Bits und speichert das Ergebnis als $b_{00}', \dots, b_{0m}'$
  1. Bob schickt die $b_{1k}'$ an Alice. Diese vergleicht sie mit ihren $a_{1k}'$. Sind für ein festes $k$ die beiden Bits gleich, behalten Alice und Bob die Bits $a_{0k}'$ und $b_{0k}'$ als Teil des Schlüssel. Andernfalls verwerfen sie die entsprechenden Bits.
  1. Bob schickt $l$ Bits des Schlüssels an Alices, welche sie mit ihren Bits vergleicht. Stellt sich heraus, dass ein Bit falsch ist, verwendne sie den Schlüssel nicht, denn sie wurden belauscht.

__Aufgabe__: Wie viele Bits des Schlüssels müssen Alice und Bob bei der obigen Lauschstrategie von Eve austauschen, um mit mindestens
* $50$ prozentiger
* $90$ prozentiger
* $99$ prozentiger
* $99{,}99$ prozentiger
Wahrscheinlichkeit festzustellen zu können, ob sie belauscht wurden?